This snippet below will open up a camera stream, show you the live stream from the camera, and provide a button to take pictures with. Click on the button to take a picture.

Please read https://medium.com/@kennethjiang/calibrate-fisheye-lens-using-opencv-333b05afa0b0 by @kennethjiang

> Calibrate fisheye lens using OpenCV — part 1

You need to follow the instructions on printing out a checkerboard pattern and take pictures of it at various angles, just like how @kennethjiang does it.

In [ ]:
# change this and run this small snippet to set the parameters correctly
master_width = 960
master_height = 720
path = "/home/jetbot/cv2_distortion_calibration"

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import cv2
import os, sys, time, datetime
from uuid import uuid1

camera = Camera.instance(width=master_width, height=master_height)

image = widgets.Image(format='jpeg', width=master_width, height=master_height)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

button_layout = widgets.Layout(width='128px', height='64px')
shutter_button = widgets.Button(description='Save Pic', button_style='success', layout=button_layout)

display(widgets.HBox([shutter_button]))

def save_snapshot():
    global path
    try:
        os.makedirs(path)
    except FileExistsError:
        pass
    image_path = os.path.join(path, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

shutter_button.on_click(lambda x: save_snapshot())

The next snippet of code is from the same tutorial by @kennethjiang, assuming all the images are saved in the "path" from the previous snippet.

In [ ]:
import cv2
import os, sys, time, datetime
assert cv2.__version__[0] == '3', 'The fisheye module requires opencv version >= 3.0.0'
import numpy as np
import glob

CHECKERBOARD = (6,9)
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + cv2.fisheye.CALIB_FIX_SKEW
calibration_flags += cv2.fisheye.CALIB_CHECK_COND # disable this if there's an exception
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

os.chdir(path) # warning, make sure path is set somewhere

images = glob.glob('*.jpg')
for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
N_OK = len(objpoints)
print("Found " + str(N_OK) + " valid images for calibration")
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )

print(" ")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")
print(" ")

When the previous snipping of code is executed, it should have output three variables: DIM, K, and D

Since we are using Jupyter, we don't need to copy them over to the next snippet, but you can still do so, or use them in other bits of code later.

The results I got for my Raspberry Pi Camera V2 with 160° wide angle lens:
```
DIM=(960, 720)
K=np.array([[460.70753878093376, 0.0, 505.24222903420747], [0.0, 461.22079408566924, 357.7314598652513], [0.0, 0.0, 1.0]])
D=np.array([[-0.016987487065241266], [-0.014653716767538133], [-0.02029760249000433], [0.016954509783374646]])
```

and for a squished square image:

```
DIM=(720, 720)
K=np.array([[346.7104965474094, 0.0, 379.81851823551904], [0.0, 462.1778768584941, 355.5060529230117], [0.0, 0.0, 1.0]])
D=np.array([[-0.04498282438799088], [0.07821348164549044], [-0.15085637279264702], [0.092845374923209]])
```

The next snippet of code is from part 2 of @kennethjiang's tutorial, https://medium.com/@kennethjiang/calibrate-fisheye-lens-using-opencv-part-2-13990f1b157f

It has a function to undistort an image, with control over the amount of "balance"

> We will need to start with understanding what ‘balance’ (also called ‘alpha’ in classic cv2 module) is, and how it impacts the dimension and shape of the undistorted image.

> the amount of pixels that get cropped off by the default un-distortion settings is too big to be acceptable.

The code snippet below will go through all the images you have taken, and apply undistortion with various amounts of balance. Run this to check your results.

In [ ]:
import numpy as np
import cv2

DIM = _img_shape[::-1] # or copy from the output of calibration
K = K                  # or copy from the output of calibration
D = D                  # or copy from the output of calibration

def undistort(img_path, balance=0.0, dim2=None, dim3=None):
    global DIM
    global K
    global D
    img = cv2.imread(img_path)
    dim1 = img.shape[:2][::-1]  #dim1 is the dimension of input image to un-distort
    assert dim1[0]/dim1[1] == DIM[0]/DIM[1], "Image to undistort needs to have same aspect ratio as the ones used in calibration"
    if not dim2:
        dim2 = dim1
    if not dim3:
        dim3 = dim1
    scaled_K = K * dim1[0] / DIM[0]  # The values of K is to scale with image dimension.
    scaled_K[2][2] = 1.0  # Except that K[2][2] is always 1.0
    # This is how scaled_K, dim2 and balance are used to determine the final K used to un-distort image. OpenCV document failed to make this clear!
    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(scaled_K, D, dim2, np.eye(3), balance=balance)
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(scaled_K, D, np.eye(3), new_K, dim3, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    return undistorted_img # this returns a np array

os.chdir(path) # make sure path is set
images = glob.glob('*.jpg')

for b in [0.0, 0.25, 0.5, 0.75, 1.0]:
    path2 = os.path.join(path, ("undistorted_bal_%03u" % (round(b * 100))))
    try:
        os.makedirs(path2)
    except FileExistsError:
        pass

    for fname in images:
        new_path = os.path.join(path2, fname)
        img = cv2.imread(fname)
        img_result = undistort(fname, balance=b)
        img_val = cv2.imencode('.jpg', img_result)[1]
        img_bytes = bytes(img_val)
        try:
            with open(new_path, 'wb') as f:
                f.write(img_bytes)
            print("finished file " + new_path)
        except Exception as ex:
            print("exception while writing \"%s\", error is: %s" % (str(ex)))
        jupimg = widgets.Image(format='jpeg', value=img_bytes, width=master_width, height=master_height)
        display(jupimg)

If we have the updated Camera class, we can test it using this notebook:

In [ ]:
camera.enable_undistort()
display(image)